In [1]:
import pandas as pd
import numpy as np
import os
import shutil

In [ ]:
PERIOD_YEAR = '2014'

In [3]:
df_interactions = pd.read_csv('../data/userid_trackid_timestamp.tsv', sep='\t')
print(df_interactions.shape)
df_interactions.head()

(252984396, 3)


,user_id,track_id,timestamp
0,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:42:38
1,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:38:53
2,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:35:08
3,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:31:23
4,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:27:38


In [4]:
df_interactions['year'] = pd.to_datetime(df_interactions['timestamp']).dt.to_period('Y')
df_interactions['month'] = pd.to_datetime(df_interactions['timestamp']).dt.to_period('M')
df_interactions = df_interactions[df_interactions['year'] == PERIOD_YEAR]
print(df_interactions.shape)
df_interactions.head()

(35069320, 5)


,user_id,track_id,timestamp,year,month
0,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:42:38,2013,2013-01
1,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:38:53,2013,2013-01
2,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:35:08,2013,2013-01
3,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:31:23,2013,2013-01
4,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:27:38,2013,2013-01


In [ ]:
# binarization: we convert the interactions to binary implicit feedback with a threshold of 2 on the interaction counts (reducing false-positive interactions)
def filter_interactions_binary(listening_history: pd.DataFrame, min_interactions: int = 2, verbose: bool = True):
    lhs_count = listening_history.value_counts(subset=['user_id', 'track_id'])
    lhs_count = lhs_count[lhs_count >= min_interactions]
    listening_history = listening_history.set_index(['user_id', 'track_id']).loc[lhs_count.index]
    listening_history = listening_history.reset_index()
    return listening_history

# filter interactions with a minimum time interval between them (alternative to binarization)
def filter_interactions_interval(listening_history: pd.DataFrame, min_interaction_interval=30):
    listening_history = listening_history.sort_values(by=['user_id', 'timestamp'])
    listening_history['timestamp'] = pd.to_datetime(listening_history['timestamp'])
    listening_history['time_diff'] = listening_history.groupby('user_id')['timestamp'].diff().dt.total_seconds().fillna(0)
    listening_history = listening_history[listening_history['time_diff'] >= min_interaction_interval]
    listening_history = listening_history.drop(columns=['time_diff'])
    return listening_history

def delete_duplicates(listening_history: pd.DataFrame):
    initial_shape = listening_history.shape
    listening_history = listening_history.drop_duplicates(subset=['user_id', 'track_id'], keep='first')
    #print(f"Deleted {initial_shape[0] - listening_history.shape[0]} duplicates")
    return listening_history

#df_interactions = filter_interactions_interval(df_interactions, min_interaction_interval=30)
df_interactions = filter_interactions_binary(df_interactions, min_interactions=2)
df_interactions = delete_duplicates(df_interactions)
print(df_interactions.shape)
df_interactions.head()

(5066229, 5)


,user_id,track_id,timestamp,year,month
0,32136,zJZUGfCgoBisXUS9,2013-04-06 14:16:43,2013,2013-04
6463,49741,wscBgW8kSMuQj55r,2013-02-28 18:01:49,2013,2013-02
12629,23954,S7RFdCOEy8yX1StR,2013-01-27 23:44:07,2013,2013-01
16263,32136,vrHa51oZkAx9JbuB,2013-11-30 13:35:52,2013,2013-11
19034,5792,L7t7nltflAuxXSKO,2013-01-01 00:50:48,2013,2013-01


In [6]:
# sparsity of dataset
n_users = df_interactions['user_id'].nunique()
n_items = df_interactions['track_id'].nunique()
n_interactions = df_interactions.shape[0]
sparsity = 1 - n_interactions / (n_users * n_items)
sparsity

0.9978609301515377

In [7]:
n_items

35039

In [8]:
n_users

67594

In [9]:
# export
df_interactions.to_csv(f'../data/userid_trackid_timestamp_{PERIOD_YEAR}.tsv', sep='\t', index=False)